In [81]:
import json
import os
import pandas as pd

In [82]:
def read_data(path):
    dirs = []
    for root, dirs, files in os.walk(path):
        for file in files:
            if file.endswith('.jsonl'):
                dirs.append(os.path.join(root, file))
    return dirs

In [115]:
# PubMedQA
pubmed_l = pd.read_json('./dataset/PubMedQA/ori_pqal.json')
pubmed_u = pd.read_json('./dataset/PubMedQA/ori_pqau.json')
pubmed_a = pd.read_json('./dataset/PubMedQA/ori_pqaa.json')

pubmed_l = pubmed_l.loc[['CONTEXTS', 'QUESTION', 'final_decision']]
pubmed_u = pubmed_u.loc[['CONTEXTS', 'QUESTION', 'LONG_ANSWER']]
pubmed_a = pubmed_a.loc[['CONTEXTS', 'QUESTION', 'final_decision']]

pubmed_l = pubmed_l.T
pubmed_u = pubmed_u.T
pubmed_a = pubmed_a.T

pubmed_l['answer_option'] = [['yes', 'no', 'maybe']] * pubmed_l.shape[0]
pubmed_u['answer_option'] = [['yes', 'no', 'maybe']] * pubmed_u.shape[0]
pubmed_a['answer_option'] = [['yes', 'no', 'maybe']] * pubmed_a.shape[0]

pubmed_l.columns = ['context', 'question', 'answer', 'answer_option']
pubmed_u.columns = ['context', 'question', 'answer', 'answer_option']
pubmed_a.columns = ['context', 'question', 'answer', 'answer_option']

pubmed_l['context'] = pubmed_l['context'].apply(lambda x: ' '.join(x))
pubmed_u['context'] = pubmed_u['context'].apply(lambda x: ' '.join(x))
pubmed_a['context'] = pubmed_a['context'].apply(lambda x: ' '.join(x))

pubmed_l.to_csv('./dataset/PubMedQA/ori_pqal.csv', index=False)
pubmed_u.to_csv('./dataset/PubMedQA/ori_pqau.csv', index=False)
pubmed_a.to_csv('./dataset/PubMedQA/ori_pqaa.csv', index=False)

In [116]:
def medmcqa_process(df):
    for i, answer in enumerate(df['answer']):
        df.loc[i, 'answer'] = df.loc[i, 'answer_option'][df.loc[i, 'answer'] - 1]

    return df

In [117]:
# MedMCQA
df_dev = pd.read_json('./dataset/MCQA/dev.json', lines=True)
df_test = pd.read_json('./dataset/MCQA/test.json', lines=True)
df_train = pd.read_json('./dataset/MCQA/train.json', lines=True)

mcqa_dev = df_dev[['subject_name', 'question']]
mcqa_dev.loc[:, 'answer_option'] = pd.Series(
    df_dev[['opa', 'opb', 'opc', 'opd']].values.tolist())
mcqa_dev.columns = ['context', 'question', 'answer_option']
mcqa_dev.loc[:, 'answer'] = df_dev.loc[:, 'cop']
medmcqa_process(mcqa_dev)

mcqa_test = df_test[['subject_name', 'question']]
mcqa_test.loc[:, 'answer_option'] = pd.Series(
    df_test[['opa', 'opb', 'opc', 'opd']].values.tolist())
mcqa_test.columns = ['context', 'question', 'answer_option']

mcqa_train = df_train[['subject_name', 'question']]
mcqa_train.loc[:, 'answer_option'] = pd.Series(
    df_train[['opa', 'opb', 'opc', 'opd']].values.tolist())
mcqa_train.columns = ['context', 'question', 'answer_option']
mcqa_train.loc[:, 'answer'] = df_train.loc[:, 'cop']
medmcqa_process(mcqa_train)

mcqa_dev.to_csv('./dataset/MCQA/dev.csv', index=False)
mcqa_test.to_csv('./dataset/MCQA/test.csv', index=False)
mcqa_train.to_csv('./dataset/MCQA/train.csv', index=False)

/var/folders/m_/93q117vx771d47222_mgd0hr0000gn/T/ipykernel_30912/3422122441.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mcqa_dev.loc[:, 'answer_option'] = pd.Series(
/var/folders/m_/93q117vx771d47222_mgd0hr0000gn/T/ipykernel_30912/3422122441.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mcqa_dev.loc[:, 'answer'] = df_dev.loc[:, 'cop']
/var/folders/m_/93q117vx771d47222_mgd0hr0000gn/T/ipykernel_30912/3422122441.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fro

In [118]:
def process(df):
    data = df[['meta_info', 'question', 'options', 'answer']]
    data = data.rename(columns={'meta_info': 'context',
                       'question': 'question', 'options': 'answer_option'})
    if (type(data['answer_option'][0]) == dict):
        data['answer_option'] = data['answer_option'].apply(
            lambda x: list(x.values()))
    if (type(data['answer'][0]) == str and len(data['answer'][0]) == 1):
        for i, answer in enumerate(data['answer']):
            data['answer'][i] = data['answer_option'][i][ord(
                data['answer'][i]) - 65]
    return data

In [119]:
# MedQA-USMLE
dirs = read_data('./dataset/data_clean/')

print(dirs)

file = [pd.read_json(file_path, lines=True) for file_path in dirs]

datasets = [(process(file), file_path) for file, file_path in zip(file, dirs)]

for (dataset, file_path) in datasets:
    dataset.to_csv(file_path[:-5] + 'csv', index=False)


['./dataset/data_clean/questions/Mainland/4_options/train.jsonl', './dataset/data_clean/questions/Mainland/4_options/test.jsonl', './dataset/data_clean/questions/Mainland/4_options/dev.jsonl']
